In [1]:
import math
import datetime
import numpy as np
import pickle

import methodsMLinterns
from customAutoencoder import ModelType, Autoencoder, ExperimentPerformance

Using TensorFlow backend.


In [2]:
stocks = ['DNB', 'NRG', 'CL', 'ANTM', 'NEE', 'PAYX', 'VAR', 'NI', 'MNST', 'JNJ', 'TGNA', 'NOV', 'FIS', 'BLK', 'HBI', 'NVDA', 'DLTR', 'MRO', 'EMN', 'AMT', 'FLR', 'IBM', 'BK', 'NFX', 'AGN', 'LRCX', 'DIS', 'LH', 'C', 'MNK']

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [4]:
features_1p4_extra =['aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'abj', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acr', 'acw', 'acx', 'acy', 'adi', 'adj', 'adl', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aex', 'aey', 'aez', 'afa', 'afj', 'afl', 'afo', 'afp', 'afq', 'afr', 'afs', 'aft', 'afu', 'afv', 'afw', 'afx', 'afy', 'afz', 'aga', 'agb', 'agc', 'agd', 'age', 'agf', 'agg', 'agh', 'agi', 'agj', 'agk', 'agl', 'agm', 'agn', 'ago', 'agp', 'agq', 'agr', 'ags', 'agt', 'agu', 'agv', 'agw', 'agx', 'agy', 'ahf', 'ahg', 'ahh', 'ahi', 'ahj', 'ahk', 'ahl', 'ahm', 'ahn', 'aho', 'zhq', 'zhr', 'zhs', 'zht', 'zhu', 'zhv', 'zhw', 'ziy', 'zjb', 'zjc', 'zjd', 'zje', 'zjf', 'zjg', 'zjh', 'zji', 'zjj', 'zjk', 'zjl', 'zjm', 'zjn', 'zjo', 'zjp', 'zjq', 'zjr', 'zjs', 'zjt', 'zju', 'zkg', 'zkl', 'zkm', 'zkn', 'zkx', 'zky', 'zla', 'zld', 'zle', 'zlf', 'zlg', 'zlh', 'zli', 'zlj', 'zlk', 'zll', 'zlm', 'zln', 'zlo', 'zlp', 'zlq', 'zlr', 'zls', 'zlt', 'zlu', 'zlv', 'zlw', 'zlx', 'zly', 'zlz', 'zma', 'zmb', 'zmc', 'zmd', 'zme', 'zmf', 'zmg', 'zmh', 'zmm', 'zmn', 'zmo', 'zmp', 'zmy', 'zna', 'znd', 'zne', 'znf', 'zng', 'znh', 'zni', 'znj', 'znk', 'znl', 'znm', 'znn', 'zno', 'znp', 'znq', 'znr', 'zns', 'znt', 'znu', 'znv', 'znw', 'znx', 'zny', 'znz', 'zoa', 'zob', 'zoc', 'zod', 'zoe', 'zof', 'zog', 'zoh', 'zoi', 'zoj', 'zok', 'zol', 'zom', 'zon', 'zou', 'zov', 'zow', 'zox', 'zoy', 'zoz', 'zpa', 'zpb', 'zpc', 'zpd', 'zpe']
features_1p4_extra_a=[f for f in features_1p4_extra if f[0]=='a']
features_1p4_extra_z=[f for f in features_1p4_extra if f[0]=='z']
assert(len(features_1p4_extra_a)+len(features_1p4_extra_z)==len(features_1p4_extra))

In [5]:
date_test_set = datetime.date(2016, 1, 1)

clf_portfolio_dic = methodsMLinterns.ClassificationPortfolio(stocks=stocks, minutes_forward=30)
clf_portfolio_dic.loadData()
clf_portfolio_dic.cleanUpData(features_1p4_extra)
clf_portfolio_dic.getTrainTestSetDate(date_test_set)

### Group together all the stocks

In [6]:
def prepareData(features):
    X_train = np.array([], dtype=np.float64).reshape(0,len(features))
    y_train = np.array([], dtype=np.float64).reshape(0,1)
    X_test = np.array([], dtype=np.float64).reshape(0,len(features))
    y_test = np.array([], dtype=np.float64).reshape(0,1)


    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = stock + str(clf_portfolio_dic.minutes_forward)
        if k==0:
            X_train, y_train = clf_portfolio_dic.X_train_dic[name][features].as_matrix(),(clf_portfolio_dic.y_train_dic[name]+1)/2
            X_test, y_test = clf_portfolio_dic.X_test_dic[name][features].as_matrix(), (clf_portfolio_dic.y_test_dic[name]+1)/2
        else:
            X_train = np.concatenate((X_train,clf_portfolio_dic.X_train_dic[name][features].as_matrix()),axis=0)
            y_train = np.concatenate((y_train,(clf_portfolio_dic.y_train_dic[name]+1)/2),axis=0)
            X_test = np.concatenate((X_test,clf_portfolio_dic.X_test_dic[name][features].as_matrix()),axis=0)
            y_test = np.concatenate((y_test,(clf_portfolio_dic.y_test_dic[name]+1)/2),axis=0)

    print(X_train.shape)
    # Transform to one hot vectors
    y_t = np.zeros((y_train.shape[0], 2))
    y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
    y_train = y_t

    y_t = np.zeros((y_test.shape[0], 2))
    y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
    y_test = y_t
    
    return X_train,y_train,X_test,y_test

### Use Keras

In [7]:
modelType = ModelType.Merged

# Define the NN architecture
features = features_1p4_extra_z
architecture = [len(features),100,30]
print(architecture)

listOfParameters = {}
listOfParameters["z_30"] = [np.asarray([0.00684]), [0.005], [256], [0.0698], ['cosine_proximity']]
lp = listOfParameters["z_30"]

weightsDirectory="_weights"

with open("pickles/CV_setsIndexes.p",'rb') as f:
    sets = pickle.load(f)
    
train_idx = sets[0][0]
val_idx = sets[0][1]

[120, 100, 30]


In [8]:
with open("pickles/CVhyperasSearchBestParameters_z_30.p",'rb') as f:
    params = pickle.load(f)
print(params)

{'d_1': 0.06989977789379659, 'd': 0.0068486252678771065, 'autoencoderLoss': 0, 'l2': 0, 'batch_size': 1}


Iterate over the different models

In [9]:
X_train, y_train, X_test, y_test = prepareData(features)
X_train, y_train, X_val, y_val = X_train[train_idx,:], y_train[train_idx,:], X_train[val_idx,:], y_train[val_idx,:]


for i in range(0,lp[0].shape[0]):
    print("\n\n\t First part:\n")
    auto = Autoencoder(architecture, modelType, weightsDirectory,
                       dropout=lp[0][i], inputNoise=lp[3][i], l2reg=lp[1][i], autoencoderLoss=lp[4][i])
    auto.buildAutoencoder()
        
    auto.fit(X_train, y_train, X_val, y_val, epochs=200, batch=lp[2][i])
    
    chkpt_0 = "%s/%s.hdf5"%(auto.wD, auto.name)
        
    auto.freezeAutoencoder(True)
        
    print("\n\n\t Second part:\n"%lp)
    auto.loadFromWeights(chkpt_0)
    auto.fit(X_train, y_train, X_val, y_val, epochs=100, batch=lp[2][i])
        

(122607, 120)


	 First part:

Epoch 00000: val_decoded_loss improved from inf to -0.00439, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00001: val_decoded_loss improved from -0.00439 to -0.00571, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00002: val_decoded_loss improved from -0.00571 to -0.00615, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00003: val_decoded_loss improved from -0.00615 to -0.00636, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00004: val_decoded_loss improved from -0.00636 to -0.00650, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00005: val_decoded_loss improved from -0.00650 to -0.00661, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00006: v

Epoch 00052: val_decoded_loss improved from -0.00739 to -0.00739, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00053: val_decoded_loss improved from -0.00739 to -0.00739, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00054: val_decoded_loss improved from -0.00739 to -0.00739, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00055: val_decoded_loss improved from -0.00739 to -0.00740, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00056: val_decoded_loss improved from -0.00740 to -0.00740, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00057: val_decoded_loss improved from -0.00740 to -0.00740, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00058: val_decoded_loss improved f

Epoch 00117: val_decoded_loss improved from -0.00746 to -0.00746, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00118: val_decoded_loss did not improve
Epoch 00119: val_decoded_loss improved from -0.00746 to -0.00746, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00120: val_decoded_loss improved from -0.00746 to -0.00746, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00121: val_decoded_loss improved from -0.00746 to -0.00746, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
Epoch 00122: val_decoded_loss did not improve
Epoch 00123: val_decoded_loss did not improve
Epoch 00124: val_decoded_loss did not improve
Epoch 00125: val_decoded_loss did not improve
Epoch 00126: val_decoded_loss did not improve
Epoch 00127: val_decoded_loss did not improve
Epoch 00128: val_decoded_loss did not 

Epoch 00002: val_logit_loss improved from 0.66924 to 0.66922, saving model to _weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f1.hdf5
Epoch 00003: val_logit_loss did not improve
Epoch 00004: val_logit_loss did not improve
Epoch 00005: val_logit_loss did not improve
Epoch 00006: val_logit_loss did not improve
Epoch 00007: val_logit_loss did not improve
Epoch 00008: val_logit_loss did not improve
Epoch 00009: val_logit_loss did not improve
Epoch 00010: val_logit_loss did not improve
Epoch 00011: val_logit_loss did not improve
Epoch 00012: val_logit_loss did not improve
Epoch 00013: val_logit_loss did not improve
Epoch 00014: val_logit_loss did not improve
Epoch 00015: val_logit_loss did not improve
Epoch 00016: val_logit_loss did not improve
Epoch 00017: val_logit_loss did not improve
Epoch 00018: val_logit_loss did not improve
Epoch 00019: val_logit_loss did not improve
Epoch 00020: val_logit_loss did not improve
Epoch 00021: val_logit_loss did not improve
Epoch 00

In [14]:
print("hello")

hello


### Test on stocks separately

In [11]:
def testOnSeparateStocks(autoList, feat):
    n = len(autoList)
    trainNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,))
    valNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,))
    testNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,))
    acc_train_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,))
    acc_val_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,))
    acc_test_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,))

    # difficulty is to recreate the validation set for each stock
    indexCount = 0
    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = "%s30"%stock
        print(k,name)
        
        # extract stock data
        x_train = np.array(clf_portfolio_dic.X_train_dic[name][feat])
        x_test = np.array(clf_portfolio_dic.X_test_dic[name][feat])
        y_train = (clf_portfolio_dic.y_train_dic[name]+1)/2
        y_test = (clf_portfolio_dic.y_test_dic[name]+1)/2
        
        # split train into val+train
        n = y_train.shape[0]
        mask = np.array([(i+indexCount in val_idx) for i in range(n)])
        x_train, y_train, x_val, y_val = x_train[~mask,:], y_train[~mask], x_train[mask,:], y_train[mask]
        indexCount += n
        assert(n==y_train.shape[0]+y_val.shape[0])

        
        # transform ys to one-hot vectors
        y_t = np.zeros((y_train.shape[0], 2))
        y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
        y_train = y_t

        y_t = np.zeros((y_val.shape[0], 2))
        y_t[np.arange(y_val.shape[0]), y_val.astype('int32')] = 1
        y_val = y_t
        
        y_t = np.zeros((y_test.shape[0], 2))
        y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
        y_test = y_t
        
        # store numbers of trades
        trainNumOfTrades[k] = y_train.shape[0]
        valNumOfTrades[k] = y_val.shape[0]
        testNumOfTrades[k] = y_test.shape[0]
    
        # compute accuracies
        for i, a in enumerate(autoList):
            y_pred_train = a.predict(x_train)
            y_pred_val = a.predict(x_val)
            y_pred_test = a.predict(x_test)
            acc_train = (100 * (np.argmax(y_pred_train,1) == np.argmax(y_train,1))).mean()
            acc_val = (100 * (np.argmax(y_pred_val,1) == np.argmax(y_val,1))).mean()
            acc_test = (100 * (np.argmax(y_pred_test,1) == np.argmax(y_test,1))).mean()
            acc_train_autoencoder[i,k] = acc_train
            acc_val_autoencoder[i,k] = acc_val
            acc_test_autoencoder[i,k] = acc_test
        
    return(acc_train_autoencoder, acc_val_autoencoder, acc_test_autoencoder,
           trainNumOfTrades, valNumOfTrades, testNumOfTrades)

In [12]:
expList = []
autoList = []

for k, features in enumerate([features_1p4_extra_z]): #features_1p4_extra_a, features_1p4_extra_z
    for j,encoding_size in enumerate([30]): #,50,70
        architecture = [len(features),100,encoding_size]
        if features == features_1p4_extra_a:
            feat = "a"
        elif features == features_1p4_extra_z:
            feat = "z"
        elif features == features_1p4_extra_a_z:
            feat = "a_z"
        lp = listOfParameters["%s_%s"%(feat,encoding_size)]
        
        # for each run
        for i in range(0,lp[0].shape[0]):
            for f in [0,1]:
                d=lp[0][i]
                l2=lp[1][i]
                batchSize=lp[2][i]
                inputNoise=lp[3][i]
                loss= "cp" if lp[4][i] == 'cosine_proximity' else "mse"
                name = "A30S_m%s_(%s,%s,%s)_d%s_in%s_r(%s,%s)_b%s_l(%s)_f%s"%(
                        modelType, len(features),100, encoding_size, d, inputNoise, 0, l2, batchSize, loss, f)
                print(name)
                e = ExperimentPerformance(
                    methodName=name, 
                    stocks=stocks, originalFeatures=features)
                auto = Autoencoder([len(features), 100, encoding_size], modelType, dropout=d,
                                    inputNoise=inputNoise,l2reg=l2,autoencoderLoss=lp[4][i])
                auto.buildAutoencoder()
                if f == 1:
                    auto.freezeAutoencoder(True)
                chkpt_0 = "%s/%s.hdf5"%(weightsDirectory,name)
                auto.loadFromWeights(chkpt_0)
                
                autoList.append(auto)
                expList.append(e)

        # Train and Test accuracy on separate stocks        
        acc_train_vec, acc_val_vec, acc_test_vec, trainNumOfTrades, valNumOfTrades, testNumOfTrades = testOnSeparateStocks(autoList,features)    
        for i, e in enumerate(expList):
            e.setTrainResults(acc_train_vec[i,:], trainNumOfTrades)
            e.setValResults(acc_val_vec[i,:], valNumOfTrades)
            e.setTestResults(acc_test_vec[i,:], testNumOfTrades)
            print("")
            print("%s | %s"%(e.getAccuracy(set='train'),e.getAccuracyMinusSigma(set='train')))
            print("%s | %s"%(e.getAccuracy(set='val'),e.getAccuracyMinusSigma(set='val')))
            print("%s | %s"%(e.getAccuracy(set='test'),e.getAccuracyMinusSigma(set='test')))

                

A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0
_weights/A30S_m1_(120,100,30)_d0.00684_in0.0698_r(0,0.005)_b256_l(cp)_f0.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_7 (Dropout)              (None, 120)           0           input_3[0][0]                    
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 100)           12100       dropout_7[0][0]                  
____________________________________________________________________________________________________
dropout_8 (Dropout)              (None, 100)      

In [13]:
print(len(expList))

2


#with open("pickles/acc_autoencoder_logit_70_z_denoising.p",'wb') as f:
    pickle.dump( expList, f, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
"_f0_" in "weightsCV_run21/test_2017-07-03_15:30:16_z_30_f0_0.hdf5"

True

In [4]:
from hyperopt import Trials, STATUS_OK, STATUS_FAIL, tpe
from keras.layers import Input, Dense, Activation, Dropout
from keras.models import Model
from keras.regularizers import l1_l2
import keras
import keras.callbacks

from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import os
import math
import datetime
import numpy as np
import pickle
import argparse

import methodsMLinterns
import customAutoencoder


features_1p4_extra_a_z =['aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'abj', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acr', 'acw', 'acx', 'acy', 'adi', 'adj', 'adl', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aex', 'aey', 'aez', 'afa', 'afj', 'afl', 'afo', 'afp', 'afq', 'afr', 'afs', 'aft', 'afu', 'afv', 'afw', 'afx', 'afy', 'afz', 'aga', 'agb', 'agc', 'agd', 'age', 'agf', 'agg', 'agh', 'agi', 'agj', 'agk', 'agl', 'agm', 'agn', 'ago', 'agp', 'agq', 'agr', 'ags', 'agt', 'agu', 'agv', 'agw', 'agx', 'agy', 'ahf', 'ahg', 'ahh', 'ahi', 'ahj', 'ahk', 'ahl', 'ahm', 'ahn', 'aho', 'zhq', 'zhr', 'zhs', 'zht', 'zhu', 'zhv', 'zhw', 'ziy', 'zjb', 'zjc', 'zjd', 'zje', 'zjf', 'zjg', 'zjh', 'zji', 'zjj', 'zjk', 'zjl', 'zjm', 'zjn', 'zjo', 'zjp', 'zjq', 'zjr', 'zjs', 'zjt', 'zju', 'zkg', 'zkl', 'zkm', 'zkn', 'zkx', 'zky', 'zla', 'zld', 'zle', 'zlf', 'zlg', 'zlh', 'zli', 'zlj', 'zlk', 'zll', 'zlm', 'zln', 'zlo', 'zlp', 'zlq', 'zlr', 'zls', 'zlt', 'zlu', 'zlv', 'zlw', 'zlx', 'zly', 'zlz', 'zma', 'zmb', 'zmc', 'zmd', 'zme', 'zmf', 'zmg', 'zmh', 'zmm', 'zmn', 'zmo', 'zmp', 'zmy', 'zna', 'znd', 'zne', 'znf', 'zng', 'znh', 'zni', 'znj', 'znk', 'znl', 'znm', 'znn', 'zno', 'znp', 'znq', 'znr', 'zns', 'znt', 'znu', 'znv', 'znw', 'znx', 'zny', 'znz', 'zoa', 'zob', 'zoc', 'zod', 'zoe', 'zof', 'zog', 'zoh', 'zoi', 'zoj', 'zok', 'zol', 'zom', 'zon', 'zou', 'zov', 'zow', 'zox', 'zoy', 'zoz', 'zpa', 'zpb', 'zpc', 'zpd', 'zpe']
features_1p4_extra_a=[f for f in features_1p4_extra_a_z if f[0]=='a']
features_1p4_extra_z=[f for f in features_1p4_extra_a_z if f[0]=='z']

stocks = ['DNB', 'NRG', 'CL', 'ANTM', 'NEE', 'PAYX', 'VAR', 'NI', 'MNST', 'JNJ', 'TGNA', 'NOV', 'FIS', 'BLK', 'HBI', 'NVDA', 'DLTR', 'MRO', 'EMN', 'AMT', 'FLR', 'IBM', 'BK', 'NFX', 'AGN', 'LRCX', 'DIS', 'LH', 'C', 'MNK']
    
date_test_set = datetime.date(2016, 1, 1)
    
print("Loading Portfolio")
    
clf_portfolio_dic = methodsMLinterns.ClassificationPortfolio(stocks=stocks, minutes_forward=30)
clf_portfolio_dic.loadData()
clf_portfolio_dic.cleanUpData(features_1p4_extra_a_z)






Using TensorFlow backend.


Loading Portfolio


In [5]:
def data():
    with open("pickles/CVhyperasSearchConfig.p",'rb') as f:
        x_train, y_train, sets, _, _, _, _ = pickle.load(f)
    return x_train, y_train, sets


def model(x_train, y_train, sets):
    if 'parameters' not in globals():
        global parameters
        parameters = []
    if 'val_loss1' not in globals():
        global val_loss1
        val_loss1 = []
    if 'val_loss2' not in globals():
        global val_loss2
        val_loss2 = []
    parameters.append(space)
    print(space)


    with open("pickles/CVhyperasSearchConfig.p",'rb') as f:
        _, _, _, feat, encod_dim, architecture, weightsDir = pickle.load(f)
    print(feat, encod_dim, architecture)
    
    autoencoderLoss = 'cosine_proximity'
    d = {{choice(numpy.arange(10,81,5)/100)}}
    l2={{choice([.0025, .005, .01, .02, .04, .08])}}
    inputNoise = 0
    batch_size = 256
    modelType = customAutoencoder.ModelType.Sequential


    log_name = datetime.datetime.today().strftime('%Y-%m-%d_%H:%M:%S')
    
    models = []
    freezed_models = []
    for train, val in sets:
        merged_model = customAutoencoder.Autoencoder(architecture, modelType, weightsDirectory=weightsDir, dropout=d, inputNoise=inputNoise, l1reg=0, l2reg=l2, autoencoderLoss=autoencoderLoss)
        merged_model.buildAutoencoder()
        models.append(merged_model)
    
    print("\n\nTraining First Model\n\n")
    val_logit_acc = 0
    for i, m in enumerate(models):
        print("\n\tFold %s\n"%i)
        checkpointName0="%s/test_%s_%s_%s_f0_%s.hdf5" % (weightsDir, log_name, feat, encod_dim, i)
        train_idx = sets[i][0]
        val_idx = sets[i][1]
        
        m.fit(x_train[train_idx,:], y_train[train_idx,:], x_train[val_idx,:], y_train[val_idx,:], epochs=2, batch=batch_size, checkpointName=checkpointName0)
            
        vl, _ = m.model1.evaluate(x_train[val_idx,:], x_train[val_idx,:], verbose=0)
        val_loss1.append(vl)
        if math.isnan(vl):
            val_loss2.append(vl)
            with open("%s/CVhyperasSearchParameters_%s_%s.p" % (weightsDir, feat, encod_dim), 'wb') as f:
                pickle.dump( [parameters,val_loss1,val_loss2], f, protocol=pickle.HIGHEST_PROTOCOL)
                return {'loss': 0, 'status': STATUS_FAIL, 'model': None}
 
        m.buildAutoencoder(trainable=False)
        m.loadFromWeights(checkpointName0)

        freezed_models.append(m)
          
        _, vlacc = m.model1.evaluate(x_train[val_idx,:], x_train[val_idx,:], verbose=0)
        val_logit_acc += vlacc
        print("VLMAE ",vlacc)



    print("\n\nTraining Freezed Model\n\n")
    val_logit_acc_freezed = 0
    for i, m in enumerate(freezed_models):
        print("\n\tFold %s\n"%i)
        checkpointName1="%s/test_%s_%s_%s_f1_%s.hdf5" % (weightsDir, log_name, feat, encod_dim, i)
        train_idx = sets[i][0]
        val_idx = sets[i][1]
        
        m.fit(x_train[train_idx,:], y_train[train_idx,:], x_train[val_idx,:], y_train[val_idx,:], epochs=1, batch=batch_size, checkpointName=checkpointName1)
        
        vl, _ = m.model.evaluate(x_train[val_idx,:], y_train[val_idx,:], verbose=0)
        val_loss2.append(vl)
        with open("%s/CVhyperasSearchParameters_%s_%s.p" % (weightsDir, feat, encod_dim), 'wb') as f:
            pickle.dump( [parameters,val_loss1,val_loss2], f, protocol=pickle.HIGHEST_PROTOCOL)
        if math.isnan(vl):
            return {'loss': 0, 'status': STATUS_FAIL, 'model': None}

        m.loadFromWeights(checkpointName1)

        _, vlacc = m.model.evaluate(x_train[val_idx,:], y_train[val_idx,:] , verbose=0)
        val_logit_acc_freezed += vlacc
        print("VLACC ",vlacc)
    
    return {'loss': -val_logit_acc_freezed, 'status': STATUS_OK, 'model': merged_model}




def hyperasSearchCV(features, encoding_dim, evals, trainPercentage=100, weightsDir="weightsCV", useValDate=False, useShuffledDays=False):
    if not os.path.isdir(weightsDir):
        os.makedirs(weightsDir)
    
    if not useValDate and not useShuffledDays:
        clf_portfolio_dic.getTrainTestSetDate(date_test_set)
    elif useValDate and trainPercentage == 80:
        clf_portfolio_dic.getTrainValTestSetDate(datetime.date(2015, 5, 1), date_test_set)
    elif useValDate and trainPercentage == 90:
        clf_portfolio_dic.getTrainValTestSetDate(datetime.date(2015, 8, 24), date_test_set)
    elif useShuffledDays and not useValDate:
        clf_portfolio_dic.getTrainValTestShuffledDaysSetDate(date_test_set, percentageTrain=trainPercentage)



    architecture = [len(features), 100, encoding_dim]
    if features == features_1p4_extra_a:
        feat = "a"
    elif features == features_1p4_extra_z:
        feat = "z"
    elif features == features_1p4_extra_a_z:
        feat = "a_z"

    print("Features %s and encoding dimension %s"%(feat,encoding_dim))

    def prepareData(features):
        x_train = np.array([], dtype=np.float32).reshape(0,len(features))
        y_train = np.array([], dtype=np.float32).reshape(0,1)
    
        for k, stock in enumerate(clf_portfolio_dic.stocks):
            name = stock + str(clf_portfolio_dic.minutes_forward)
            if k==0:
                x_train, y_train = clf_portfolio_dic.X_train_dic[name][features].as_matrix(),(clf_portfolio_dic.y_train_dic[name]+1)/2
            else:
                x_train = np.concatenate((x_train,clf_portfolio_dic.X_train_dic[name][features].as_matrix()),axis=0)
                y_train = np.concatenate((y_train,(clf_portfolio_dic.y_train_dic[name]+1)/2),axis=0)

        y_t = np.zeros((y_train.shape[0], 2))
        y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
        y_train = y_t
    
        return x_train,y_train



    print("Loading Data")
    assert(trainPercentage in [80,90,100])
    with open("pickles/CV_setsIndexes_%sp.p"%trainPercentage,'rb') as f:
        sets = pickle.load(f)
   
    X_train, y_train = prepareData(features)

    with open("pickles/CVhyperasSearchConfig.p",'wb') as f:
        pickle.dump( [X_train, y_train, sets, feat, encoding_dim, architecture, weightsDir], f, protocol=pickle.HIGHEST_PROTOCOL)

    best_run, _ = optim.minimize(model=model,
                                 data=data,
                                 algo=tpe.suggest,
                                 max_evals=evals,
                                 trials=Trials())

    with open("%s/CVhyperasSearchBestParameters_%s_%s.p" % (weightsDir, feat, encoding_dim),'wb') as f:
        pickle.dump( best_run, f, protocol=pickle.HIGHEST_PROTOCOL)

    with open("%s/CVhyperasSearchParameters_%s_%s.p" % (weightsDir, feat, encoding_dim),'rb') as f:
        parameters,val_loss1,val_loss2 = pickle.load(f)
    print("----------")
    for p in parameters:
        print(p)
    print("----------")
    for v in val_loss1:
        print(v)
    print("----------")
    for v in val_loss2:
        print(v)




hyperasSearchCV(features_1p4_extra_z, 30, evals=2, trainPercentage=90, weightsDir="weightsCV_run21", useShuffledDays=True)


/usr/local/lib/python3.5/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Features z and encoding dimension 30
Loading Data
/usr/local/lib/python3.5/site-packages/hyperas/optim.py
/usr/local/lib/python3.5/site-packages/hyperas/optim.py
/usr/local/lib/python3.5/site-packages/hyperas/optim.py
<ipython-input-5-471320de4d80>
<ipython-input-5-471320de4d80>
/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py
/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py
/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py
/usr/local/lib/python3.5/site-packages/ipykernel/zmqshell.py
/usr/local/lib/python3.5/site-packages/ipykernel/ipkernel.py
/usr/local/lib/python3.5/site-packages/ipykernel/kernelbase.py
/usr/local/lib/python3.5/site-packages/ipykernel/kernelbase.py
/usr/local/lib/python3.5/site-packages/ipykernel/kernelbase.py
/usr/local/lib/python3.5/site-packages/tornado/stack_context.py
/usr/local/lib/python3.5/site-packages/zmq/eventloop/zmqstream.py
/usr/local/lib/python3.5/site-packages/zmq/eventloop/zmqstream.

FileNotFoundError: [Errno 2] No such file or directory: '/Users/clement/Dropbox (Personal)/ArrowInterns/clement/<ipython-input-5-471320de4d80>'